In [7]:
# Kirjastojen tuonti
import geopandas as gpd
import urllib.parse
import json
import pandas as pd
from dash import dcc, html, Dash
from dash.dependencies import Output, Input
import plotly.express as px

--- Konfiguraatio ja Muuttumattomat Tiedot ---

In [1]:
# 23 kuntaa, joiden rajat haetaan WFS:stä (FIXED: Konnevesi)
canonical_cities = [
    "Iisalmi", "Keitele", "Kiuruvesi", "Kuopio", "Lapinlahti", 
    "Leppävirta", "Pielavesi", "Siilinjärvi", "Sonkajärvi", "Suonenjoki", 
    "Tervo", "Varkaus", "Vesanto", "Rautavaara", "Vieremä", 
    "Joensuu", "Joroinen", "Pieksämäki", 
    "Hankasalmi", "Konnevesi", "Rautalampi", 
    "Laukaa", "Äänekoski"
]

In [2]:
# Vanhojen kuntien ja nykyisten kuntien nimenmuutokset (koska stream-data käyttää vanhoja nimiä)
location_map = {
    'Nilsiä': 'Kuopio', 
    'Tahkovuori': 'Kuopio', 
    'Juankoski': 'Kuopio', 
    'Varpaisjärvi': 'Lapinlahti'
}

In [ ]:
# --- GeoData haku (Kartan rajat) ---
cql_values = ", ".join(f"'{name}'" for name in canonical_cities)
cql_filter = f"name IN ({cql_values})"
encoded_cql_filter = urllib.parse.quote(cql_filter)
base_url = "https://geo.stat.fi/geoserver/tilastointialueet/wfs"
url = (
    f"{base_url}?service=WFS&version=2.0.0&request=GetFeature"
    f"&typeName=tilastointialueet:kunta1000k_2025&outputFormat=application/json"
    f"&CQL_FILTER={encoded_cql_filter}"
)

In [ ]:
print("Fetching municipal boundaries...")
gdf_filtered = gpd.read_file(url)
gdf_filtered['mention_count'] = 0 # Alustava laskuri kartan väritystä varten

# UUSI RIVI: Varmista, että GeoJSONin ominaisuudet sisältävät nimen
gdf_filtered['name_id'] = gdf_filtered['name']

In [8]:
# --- Stream Datan Lataus (Simuloitu) ---
try:
    # Varmista, että tiedostopolku on oikein
    with open('E:\projects\python\data-pipeline\data\processed\outage_data.json ', 'r', encoding='utf-8') as f:
        STREAM_DATA = json.load(f)
except FileNotFoundError:
    print("FATAL ERROR: outage_data.json not found. Exiting.")
    exit()

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\MacroHard\AppData\Local\Temp\ipykernel_27324\1946483783.py:4: SyntaxWarning: invalid escape sequence '\p'
  with open('E:\projects\python\data-pipeline\data\processed\outage_data.json ', 'r', encoding='utf-8') as f:


In [ ]:
# Globaali tila Dash-sovelluksen käyttöön
stream_state = {
    'index': 0,
    'gdf_state': gdf_filtered.copy() 
}
print(f"GeoData loaded with {len(gdf_filtered)} features. Stream size: {len(STREAM_DATA)} events.")

In [9]:
# Initialize JupyterDash app
app = Dash(__name__)

In [10]:
# --- Dashboard Asettelu (Layout) ---
app.layout = html.Div([
    html.H1("Real-Time Outage Dashboard", style={'textAlign': 'center'}),
    
    # Ylätason sisältöalue (Map ja Bar Chart rinnakkain)
    html.Div([
        # 1. Kartta (Choropleth Map)
        html.Div(dcc.Graph(id='live-map'), 
                 style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
        
        # 2. Palkkikaavio (Bar Chart)
        html.Div(dcc.Graph(id='live-bar-chart'), 
                 style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
    ], style={'display': 'flex'}), # Käytetään flexboxia rinnakkain asetteluun
    
    # 3. Teksti-ilmoitus (Latest Event)
    html.Div([
        html.H3("Viimeisin tapahtuma:", style={'margin-top': '20px'}),
        html.Div(id='latest-event-text', style={'fontSize': '18px', 'padding': '10px', 'backgroundColor': '#f5f5f5'}),
    ]),
    
    # 4. Aikasarjakaavio (Placeholder for 4th element - now showing cumulative events)
    html.Div(dcc.Graph(id='cumulative-events-chart')),
    
    # Interval Component: Syke, joka käynnistää päivityksen 250ms välein
    dcc.Interval(
        id='interval-component',
        interval=250, # 250 milliseconds (0.25s)
        n_intervals=0
    )
])

In [11]:
# --- DASH CALLBACK: Päivitysfunktio ---
@app.callback(
    [Output('live-map', 'figure'),
     Output('live-bar-chart', 'figure'),
     Output('latest-event-text', 'children'),
     Output('cumulative-events-chart', 'figure')],
    Input('interval-component', 'n_intervals')
)
def update_dashboard(n):
    global stream_state
    
    # 1. Hae nykyinen tietue ja siirrä indeksiä eteenpäin
    i = stream_state['index']
    
    # Jos stream loppuu, aloitetaan alusta
    if i >= len(STREAM_DATA):
        stream_state['index'] = 0 
        i = 0
        # Nollaa kartan värit uutta kierrosta varten
        stream_state['gdf_state']['mention_count'] = 0
    
    new_entry_raw = STREAM_DATA[i]
    
    # 2. Käsittele sijainti (mapataan vanhat nimet)
    location_name_raw = new_entry_raw.get('location')
    location_name = location_map.get(location_name_raw, location_name_raw)

    # 3. Päivitä globaali GeoDataFrame-tila
    if location_name in stream_state['gdf_state']['name'].values:
        # Kasvata mention_count-laskuria -> kartan väri tummenee
        stream_state['gdf_state'].loc[
            stream_state['gdf_state']['name'] == location_name, 
            'mention_count'
        ] += 1
    # Huom: Muutoin ohitetaan päivitys, jos sijaintia ei löydy

    stream_state['index'] += 1

    # --- ELEMENT 1: Choropleth Map ---
    map_fig = px.choropleth( # KÄYTÄ px.choropleth
        # HUOM: Käytämme GeoDataFramea suoraan 'data_frame'-argumentilla
        data_frame=stream_state['gdf_state'], 
        
        # Määritellään GeoJSON-geometriat
        geojson=stream_state['gdf_state'].geometry.__geo_interface__, 
        
        # Määritellään, miten GeoJSONin feature-ID (kuntanimi) yhdistetään dataan:
        locations='name', 
        featureidkey="properties.name", 
        
        # Väritys stream-datan perusteella
        color='mention_count',
        color_continuous_scale="YlOrRd",
        
        # Aseta kartan laajuus ja otsikko
        scope="europe", 
        title=f"Kuntien mainintojen lukumäärä (Tapahtumat yhteensä: {i+1})",
        labels={'mention_count': 'Tapahtumien lkm'}
    )
    
    # KORJAUS: Aseta keskipiste ja zoom update_geos-metodilla (oikea tapa choropleth-kartalle)
    map_fig.update_geos(
        fitbounds="locations",  # Zoom in on the regions
        visible=False,
        projection_type="mercator"  # Specify projection if needed
    )
    
    map_fig.update_layout(
        margin={"r":0,"t":40,"l":0,"b":0},
        # Voit asettaa taustavärin tässä, jos haluat 'mapbox' -tyylin taustan.
    )
    # --- ELEMENT 2: Bar Chart (Top 5) ---
    bar_df = stream_state['gdf_state'].sort_values('mention_count', ascending=False).head(5)
    bar_fig = px.bar(
        bar_df, 
        x='name', 
        y='mention_count', 
        title="  5 eniten mainittua kuntaa"
    )

    # --- ELEMENT 3: Text Display ---
    latest_event_text = (
        f"Sijainti: **{location_name_raw}** | Päivämäärä: {new_entry_raw['day']}.{new_entry_raw['month']}.{new_entry_raw['year']} "
        f"| Kesto: {new_entry_raw['time_start']} - {new_entry_raw['time_end']}"
    )
    
    # --- ELEMENT 4: Cumulative Events Chart (Placeholder) ---
    # Luodaan yksinkertainen viivakaavio kumulatiivisesta tapahtumamäärästä
    cumulative_events_fig = px.line(
        x=list(range(i+1)), 
        y=[n for n in range(i+1)], 
        title="Kumulatiivinen tapahtumamäärä"
    )

    return map_fig, bar_fig, latest_event_text, cumulative_events_fig

In [ ]:
# Aja sovellus VS Code Jupyter Notebookissa
app.run(port=8050)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run.

[2025-11-30 20:49:24,342] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "e:\projects\python\data-pipeline\venv\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\projects\python\data-pipeline\venv\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\projects\python\data-pipeline\venv\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\projects\python\data-pipeline\venv\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\projects\pytho